In [255]:
import os
import sys
import subprocess
print(sys.version)
print(sys.executable)

sdata1 = '''
4 2
1 2
3 2
'''
sdata2 = '''
4 3
1 2
3 2
4 3
'''

sdata3 = '''
6 10
1 1
2 2
3 3
4 4
5 1
1 5
3 2
4 3
2 1
6 6
'''

pname = 'count-comps.py'
res = subprocess.run([sys.executable, pname], stderr=subprocess.STDOUT,
                     stdout=subprocess.PIPE, input=bytes(sdata1.lstrip(), 'utf8'))
print(res.stdout)

res = subprocess.run([sys.executable, pname], stderr=subprocess.STDOUT,
                     stdout=subprocess.PIPE, input=bytes(sdata2.lstrip(), 'utf8'))
print(res.stdout)

res = subprocess.run([sys.executable, pname], stderr=subprocess.STDOUT,
                     stdout=subprocess.PIPE, input=bytes(sdata3.lstrip(), 'utf8'))
print(str(res.stdout))

3.6.3 |Anaconda, Inc.| (default, Oct 15 2017, 03:27:45) [MSC v.1900 64 bit (AMD64)]
C:\Python\Anaconda3\python.exe
b'2\r\n'
b'1\r\n'
b'2\r\n'


In [164]:
%reset -sf

In [168]:
sdata1 = '''
4 2
1 2
3 2
'''
sdata2 = '''
4 3
1 2
3 2
4 3
'''

sdata3 = '''
0 0
1 3
5 2
4 6
4 1
'''

sdata = sdata2
rdata = [[int(x) for x in s.strip().split()] for s in sdata.strip().splitlines()]
nv, ne = rdata[0]
edges = rdata[1:]

#############################
ne = len(edges)
if ne and nv <= 0:
    nv = max([max(a, b) for a, b in edges])
edges = [(a-1, b-1) for a, b in edges]

nc = 0
comps = [0] * nv
for i in range(ne):
    a, b = e = edges[i]
    if comps[a] > 0:
        comps[b] = comps[a]
    elif comps[b] > 0:
        comps[a] = comps[b]
    else:
        nc += 1
        comps[a] = comps[b] = nc
        for j in range(i+1, ne):
            a, b = edges[j]
            if a in e or b in e:
                comps[a] = comps[b] = nc
for a in range(nv):
    if comps[a] == 0:
        nc += 1
        comps[a] = nc
#############################

print('Nedges:', ne)
print('Nverts:', nv)
print('edges:', edges)
print('comps:', comps)
print('nc:', nc)

Nedges: 3
Nverts: 4
edges: [(0, 1), (2, 1), (3, 2)]
comps: [1, 1, 1, 1]
nc: 1


In [225]:
sdata1 = '''
4 2
1 2
3 2
'''
sdata2 = '''
4 3
1 2
3 2
4 3
'''

sdata3 = '''
 0
6 -1
1 1
2 2
3 3
4 4
5 1
1 5
3 2
4 3
2 1
'''

sdata4 = '''
0 0
1 1
1 1
1 1
1 1
'''

sdata = sdata3
rdata = [[int(x) for x in s.strip().split()] for s in sdata.strip().splitlines()]
nv, ne = rdata[0]
edges = rdata[1:]

#############################
ne = len(edges)
if edges and nv <= 0:
    nv = max([max(a,b) for a,b in edges])
assert(all([1<=a<=nv and 1<=b<=nv for a,b in edges]))
edges = [(a-1, b-1) for a,b in edges]

verts = [[] for _ in range(nv)]
for a,b in edges:
    if b not in verts[a]:
        verts[a].append(b)
    if a not in verts[b]:
        verts[b].append(a)
for va in verts:  va.sort()
        
nc = 0
comps = [0] * nv
for a, va in enumerate(verts):
    if comps[a] <= 0:
        nc += 1
        comps[a] = nc
    for b in va:
        comps[b] = comps[a]
#############################

print('Nedges:', ne)
print('Nverts:', nv)
print('edges:', edges)
print('verts:', verts)
print('comps:', comps)
print('nc:', nc)

Nedges: 10
Nverts: 6
edges: [(5, 5), (0, 0), (1, 1), (2, 2), (3, 3), (4, 0), (0, 4), (2, 1), (3, 2), (1, 0)]
verts: [[0, 1, 4], [0, 1, 2], [1, 2, 3], [2, 3], [0], [5]]
comps: [1, 1, 1, 1, 1, 2]
nc: 2


In [252]:
sdata1 = '''
4 2
1 2
3 2
'''
sdata2 = '''
4 3
1 2
3 2
4 3
'''

sdata3 = '''
6 0
6 6
1 1
2 2
3 3
4 4
5 1
1 5
3 2
4 3
2 1
'''

sdata4 = '''
0 0
1 1
1 1
1 1
1 1
'''

sdata = sdata3
rdata = [[int(x) for x in s.strip().split()] for s in sdata.strip().splitlines()]
nv, ne = rdata[0]
edges = rdata[1:]

ne = len(edges)
if edges and nv <= 0:
    nv = max([max(a,b) for a,b in edges])
assert(all([1<=a<=nv and 1<=b<=nv for a,b in edges]))
edges = [(a-1, b-1) for a,b in edges]

#############################
verts = [set() for _ in range(nv)]
for a,b in edges:
    verts[a].add(b)
    verts[b].add(a)

def dfs(v):
    visited[v] = True
    for w in verts[v]:
        if not visited[w]:
            dfs(w)

visited = [False] * nv
nc = 0
while 1:
    try:
        i = visited.index(False)
    except ValueError:
        break
    nc += 1
    dfs(i)
#############################

print('Nedges:', ne)
print('Nverts:', nv)
print('edges:', edges)
print('verts:', verts)
print('nc:', nc)

Nedges: 10
Nverts: 6
edges: [(5, 5), (0, 0), (1, 1), (2, 2), (3, 3), (4, 0), (0, 4), (2, 1), (3, 2), (1, 0)]
verts: [{0, 1, 4}, {0, 1, 2}, {1, 2, 3}, {2, 3}, {0}, {5}]
nc: 2
